In [10]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

# Load and preprocess images
img1_left = cv.imread("samples/nb_front.png", cv.IMREAD_GRAYSCALE)
img2_left = cv.imread("samples/nb_left.png", cv.IMREAD_GRAYSCALE)

# Create SIFT_left detector with custom parameters for more keypoints
sift_left = cv.SIFT_create(
    nfeatures=0,  # Maximum number of features (0 = unlimited)
    nOctaveLayers=3,
    contrastThreshold=0.002,  # Lower threshold = more keypoints (default is 0.04)
    edgeThreshold=16,  # Higher threshold = more keypoints (default is 10)
    sigma=1.2,
)

# find the keypoints and descriptors with SIFT_left
kp1_left, des1_left = sift_left.detectAndCompute(img1_left, None)
kp2_left, des2_left = sift_left.detectAndCompute(img2_left, None)

print(f"Keypoints in image 1: {len(kp1_left)}")
print(f"Keypoints in image 2: {len(kp2_left)}")

# FLANN_left parameters - slightly modified for better performance
FLANN_left_INDEX_KDTREE = 5
index_params_left = dict(algorithm=FLANN_left_INDEX_KDTREE, trees=4)  # Increased from 5
search_params_left = dict(checks=120)  # Increased from 50

flann_left = cv.FlannBasedMatcher(index_params_left, search_params_left)

# Cross-check matching for better results
matches1to2_left = flann_left.knnMatch(des1_left, des2_left, k=2)
matches2to1_left = flann_left.knnMatch(des2_left, des1_left, k=2)

# Apply ratio test with a higher threshold (0.85 instead of 0.8)
ratio_thresh_left = 0.88

# First direction (1 to 2)
good_matches1to2_left_left = []
pts1_temp_left = []
pts2_temp_left = []

for i, (m, n) in enumerate(matches1to2_left):
    if m.distance < ratio_thresh_left * n.distance:
        good_matches1to2_left_left.append(m)
        pts1_temp_left.append(kp1_left[m.queryIdx].pt)
        pts2_temp_left.append(kp2_left[m.trainIdx].pt)

good_matches2to1_left = []
for i, (m, n) in enumerate(matches2to1_left):
    if m.distance < ratio_thresh_left * n.distance:
        good_matches2to1_left.append(m)

pts1_left = pts1_temp_left
pts2_left = pts2_temp_left

pts1_left = np.int32(pts1_left)
pts2_left = np.int32(pts2_left)

print(f"Number of matches before RANSAC: {len(pts1_left)}")

# Use RANSAC with a more lenient threshold to keep more matches
F, mask = cv.findFundamentalMat(
    pts1_left,
    pts2_left,
    method=cv.FM_RANSAC,
    ransacReprojThreshold=20.0,  # Default is 3.0, higher = more matches
    confidence=0.99,
)  # Higher confidence = more iterations

# We select only inlier points
pts1_left = pts1_left[mask.ravel() == 1]
pts2_left = pts2_left[mask.ravel() == 1]


# Rest of your code remains the same
def drawlines(img1_left, img2_left, lines, pts1_left, pts2_left):
    r, c = img1_left.shape
    img1_left = cv.cvtColor(img1_left, cv.COLOR_GRAY2BGR)
    img2_left = cv.cvtColor(img2_left, cv.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1_left, pts2_left):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1_left = cv.line(img1_left, (x0, y0), (x1, y1), color, 1)
        img1_left = cv.circle(img1_left, tuple(pt1), 5, color, -1)
        img2_left = cv.circle(img2_left, tuple(pt2), 5, color, -1)
    return img1_left, img2_left


# Find epilines corresponding to points in right image and drawing lines on left image
lines1 = cv.computeCorrespondEpilines(pts2_left.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img5_left, img6_left = drawlines(img1_left, img2_left, lines1, pts1_left, pts2_left)

# Find epilines corresponding to points in left image and drawing lines on right image
lines2 = cv.computeCorrespondEpilines(pts1_left.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img3_left, img4_left = drawlines(img2_left, img1_left, lines2, pts2_left, pts1_left)

plt.figure(figsize=(30, 18))
plt.subplot(121), plt.imshow(img5_left)
plt.subplot(122), plt.imshow(img3_left)
plt.show()

print(f"Final number of point matches: {len(pts1_left)}")

Keypoints in image 1: 5170
Keypoints in image 2: 3579
Number of matches before RANSAC: 1347
Final number of point matches: 870


In [11]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

# Load and preprocess images
img1_right = cv.imread("samples/nb_front.png", cv.IMREAD_GRAYSCALE)
img2_right = cv.imread("samples/nb_right.png", cv.IMREAD_GRAYSCALE)

# Create SIFT_right detector with custom parameters for more keypoints
sift_right = cv.SIFT_create(
    nfeatures=0,  # Maximum number of features (0 = unlimited)
    nOctaveLayers=3,
    contrastThreshold=0.002,  # Lower threshold = more keypoints (default is 0.04)
    edgeThreshold=16,  # Higher threshold = more keypoints (default is 10)
    sigma=1.2,
)

# find the keypoints and descriptors with SIFT_right
kp1_right, des1_right = sift_right.detectAndCompute(img1_right, None)
kp2_right, des2_right = sift_right.detectAndCompute(img2_right, None)

print(f"Keypoints in image 1: {len(kp1_right)}")
print(f"Keypoints in image 2: {len(kp2_right)}")

# FLANN_right parameters - slightly modified for better performance
FLANN_right_INDEX_KDTREE = 5
index_params_right = dict(
    algorithm=FLANN_right_INDEX_KDTREE, trees=4
)  # Increased from 5
search_params_right = dict(checks=120)  # Increased from 50

flann_right = cv.FlannBasedMatcher(index_params_right, search_params_right)

# Cross-check matching for better results
matches1to2_right = flann_right.knnMatch(des1_right, des2_right, k=2)
matches2to1_right = flann_right.knnMatch(des2_right, des1_right, k=2)

# Apply ratio test with a higher threshold (0.85 instead of 0.8)
ratio_thresh_right = 0.88

# First direction (1 to 2)
good_matches1to2_right = []
pts1_temp_right = []
pts2_temp_right = []

for i, (m, n) in enumerate(matches1to2_right):
    if m.distance < ratio_thresh_right * n.distance:
        good_matches1to2_right.append(m)
        pts1_temp_right.append(kp1_right[m.queryIdx].pt)
        pts2_temp_right.append(kp2_right[m.trainIdx].pt)

good_matches2to1_right_right = []
for i, (m, n) in enumerate(matches2to1_right):
    if m.distance < ratio_thresh_right * n.distance:
        good_matches2to1_right_right.append(m)

pts1_right = pts1_temp_right
pts2_right = pts2_temp_right

pts1_right = np.int32(pts1_right)
pts2_right = np.int32(pts2_right)

print(f"Number of matches before RANSAC: {len(pts1_right)}")

# Use RANSAC with a more lenient threshold to keep more matches
F, mask = cv.findFundamentalMat(
    pts1_right,
    pts2_right,
    method=cv.FM_RANSAC,
    ransacReprojThreshold=20.0,  # Default is 3.0, higher = more matches
    confidence=0.99,
)  # Higher confidence = more iterations

# We select only inlier points
pts1_right = pts1_right[mask.ravel() == 1]
pts2_right = pts2_right[mask.ravel() == 1]


# Rest of your code remains the same
def drawlines(img1_right, img2_right, lines, pts1_right, pts2_right):
    r, c = img1_right.shape
    img1_right = cv.cvtColor(img1_right, cv.COLOR_GRAY2BGR)
    img2_right = cv.cvtColor(img2_right, cv.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1_right, pts2_right):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1]])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])
        img1_right = cv.line(img1_right, (x0, y0), (x1, y1), color, 1)
        img1_right = cv.circle(img1_right, tuple(pt1), 5, color, -1)
        img2_right = cv.circle(img2_right, tuple(pt2), 5, color, -1)
    return img1_right, img2_right


# Find epilines corresponding to points in right image and drawing lines on left image
lines1 = cv.computeCorrespondEpilines(pts2_right.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img5_right, img6_right = drawlines(
    img1_right, img2_right, lines1, pts1_right, pts2_right
)

# Find epilines corresponding to points in left image and drawing lines on right image
lines2 = cv.computeCorrespondEpilines(pts1_right.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img3_right, img4_right = drawlines(
    img2_right, img1_right, lines2, pts2_right, pts1_right
)

plt.figure(figsize=(30, 18))
plt.subplot(121), plt.imshow(img5_right)
plt.subplot(122), plt.imshow(img3_right)
plt.show()

print(f"Final number of point matches: {len(pts1_right)}")

Keypoints in image 1: 5170
Keypoints in image 2: 3602
Number of matches before RANSAC: 1341
Final number of point matches: 884


In [ ]:
import numpy as np
import cv2 as cv
from math import sin, cos, radians


def triangulate_3D_points(pts1, pts2, is_left=1):
    # Camera parameters
    focal_length_mm = 50
    sensor_width_mm = 36
    sensor_height_mm = 20.25
    image_width_px = 1920
    image_height_px = 1080

    # Calculate focal lengths in pixels
    fx = (focal_length_mm / sensor_width_mm) * image_width_px
    fy = (focal_length_mm / sensor_height_mm) * image_height_px

    # Principal point (assumed at image center)
    cx = image_width_px / 2
    cy = image_height_px / 2

    # Intrinsic matrix
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

    if is_left:
        # Extrinsic parameters
        R1 = cv.Rodrigues(np.deg2rad([15, 0, 0]))[0]
        t1 = np.array([[0], [0], [-1]])

        R2 = cv.Rodrigues(np.deg2rad([-30, 0, 0]))[0]
        t2 = np.array([[-0.5], [0], [-1]])
    else:
        R1 = cv.Rodrigues(np.deg2rad([-15, 0, 0]))[0]
        t1 = np.array([[0], [0], [-1]])

        R2 = cv.Rodrigues(np.deg2rad([30, 0, 0]))[0]
        t2 = np.array([[0.5], [0], [-1]])

    # Projection matrices
    P1 = K @ np.hstack((R1, t1))
    P2 = K @ np.hstack((R2, t2))

    # Ensure matching number of points
    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)
    if pts1.shape[0] != pts2.shape[0]:
        raise ValueError("pts1 and pts2 must have the same number of points.")

    # Convert to homogeneous 2D
    pts1_hom = cv.convertPointsToHomogeneous(pts1).reshape(-1, 3)
    pts2_hom = cv.convertPointsToHomogeneous(pts2).reshape(-1, 3)

    # Triangulate and convert to 3D
    points_4D = cv.triangulatePoints(P1, P2, pts1_hom[:, :2].T, pts2_hom[:, :2].T)
    points_3D = (points_4D[:3] / points_4D[3]).T  # shape (N, 3)

    if is_left:
        points_3D[:, 0] -= 0.25
    else:
        points_3D[:, 0] += 0.25

    angle_deg = -22.5 if is_left else 22.5
    # Define rotation angle (in degrees or radians)
    theta = np.deg2rad(angle_deg)  # Replace with your desired angle

    # Rotation matrix around y-axis
    Ry = np.array(
        [
            [np.cos(theta), 0, np.sin(theta)],
            [0, 1, 0],
            [-np.sin(theta), 0, np.cos(theta)],
        ]
    )

    # Apply rotation
    points_3D = (Ry @ points_3D.T).T

    return points_3D

In [13]:
points_3D_left = triangulate_3D_points(pts1_left, pts2_left, 1)
points_3D_right = triangulate_3D_points(pts1_right, pts2_right, 0)

print(points_3D_left.shape)
print(points_3D_right.shape)

(870, 3)
(884, 3)


In [14]:
# Ensure C-contiguous arrays and flatten
points_3D_left_contig = np.ascontiguousarray(points_3D_left).reshape(-1, 3)
points_3D_right_contig = np.ascontiguousarray(points_3D_right).reshape(-1, 3)

# Define structured dtype for 3D comparison
dt3 = np.dtype(
    [
        ("x", points_3D_left.dtype),
        ("y", points_3D_left.dtype),
        ("z", points_3D_left.dtype),
    ]
)

# View as structured arrays
left_structured = points_3D_left_contig.view(dt3).reshape(-1)
right_structured = points_3D_right_contig.view(dt3).reshape(-1)

# Find common and unique elements
common = np.intersect1d(left_structured, right_structured)
unique_left = np.setdiff1d(left_structured, common)
unique_right = np.setdiff1d(right_structured, common)

# Convert back to standard 3D arrays
merged_structured = np.concatenate((unique_left, unique_right, common))
merged_3D = merged_structured.view(points_3D_left.dtype).reshape(-1, 3)
common_3D = common.view(points_3D_left.dtype).reshape(-1, 3)

print(unique_left.shape)
print(unique_right.shape)
print(common.shape)
print(merged_3D.shape)

(832,)
(850,)
(0,)
(1682, 3)


In [ ]:
# import matplotlib.pyplot as plt

# merged_3D = merged_3D[merged_3D[:, 2] <= 0.08]
# merged_3D = merged_3D[merged_3D[:, 2] >= 0]

# # Plot only x and y (first two elements)
# plt.figure(figsize=(8, 6))
# plt.scatter(merged_3D[:, 0], merged_3D[:, 1], c="green", label="Common", s=5)

# plt.legend()
# plt.xlabel("X")
# plt.ylabel("Y")
# plt.title("2D Projection of 3D Points")
# plt.grid(True)
# plt.axis("equal")
# plt.show()

In [23]:
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# # Convert back to 3D arrays for plotting
unique_left_3D = unique_left.view(points_3D_left.dtype).reshape(-1, 3)
unique_right_3D = unique_right.view(points_3D_right.dtype).reshape(-1, 3)
common_3D = common.view(points_3D_left.dtype).reshape(-1, 3)

p3d = merged_3D
p3d = p3d[p3d[:, 2] <= 0.04]
p3d = p3d[p3d[:, 2] >= 0]

# Create 3D figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for 3D points
ax.scatter(p3d[:, 0], p3d[:, 1], p3d[:, 2], c='green', label='Merged', s=5)

# Labels and settings
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Point Cloud')
ax.legend()
plt.grid(True)
plt.show()